In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import scipy
from scipy import stats as stats
from copy import copy as copy

from openpyxl import load_workbook
from openpyxl.worksheet.table import Table
# import seaborn as sns


In [ ]:
from functions.plotting import(
    volcanoPlotpVal,
    volcanoPlot)

In [ ]:
writeOutput = False

In [ ]:
# subfolder = input("Enter the name of the working folder (Must be same level as code folder)")
subfolder = "Larisa_Spheroids"
print(subfolder)

os.chdir("../" + subfolder)


In [ ]:
os.getcwd()

In [ ]:
# read in paths from config file
configDict = {
    'rootDir': '',
    'initialDataPath' : '',
    'QCDataPath' : '',
    'labWorksheet01Path':'',
    'sampleInfoFile' : '',
    'projectName':'',
}

with open('./config.txt','r') as f:
    lines = f.readlines()
    for line in lines:
        if not line.startswith('#'):
            line = line.strip()
            fields = line.split(':')

            if fields[0].strip()=='initialDataPath':
                configDict[fields[0].strip()] = fields[1].strip().strip('\'')
            else:
                configDict[fields[0].strip()] = fields[1].strip().strip('\'')
## ToDo: Add checks to ensure that minimal fields have been populated. Raise errors or warnings

In [ ]:
configDict

# Define functions

In [ ]:
# Functinos have been moved out to .py files.

# Run Analysis

In [ ]:
# ToDo:
# add sample info file details into config file in previous notebook
# sampleInfo_with_Wells.csv

In [ ]:
# ToDo: Should initial filtering be done using none-mean-HKGeoMean normalised data. Using a type of background subtraction for filtering would bias towards removing noisy or low expressing samples. Removing these may give more accurate estimates of real probe values.

# Read-in nanostring normalised data

In [ ]:


normDir = os.path.join(configDict['rootDir'], 'Normalisation')

normDir

QCDataFile = [f for f in os.listdir(normDir) if (f.startswith('QC') and f.endswith('RUV.csv'))][0]

QCDataFile

# tempDF



### Sort samples and probes 

In [ ]:

# tempDF1 = pd.read_csv(os.path.join(normDir, 'NSNorm',files[0]), index_col=0)
# v = tempDF1.mean(axis=1).sort_values().index

# sampleOrder = sorted(tempDF1.columns)

QCDataDF = pd.read_csv(os.path.join(normDir, QCDataFile), index_col=0)
QCDataDF.columns = [x.replace(' ','.') for x in QCDataDF.columns]
QCDataDF.columns = [x.replace('-','.') for x in QCDataDF.columns]


codeClass = QCDataDF.loc[:,'Code.Class']

probeOrder = QCDataDF.index[8:]
sampleOrder = QCDataDF.columns[1:]

# sampleOrder

# Read-in sampleInfo

In [ ]:
sampleInfo = pd.read_csv(os.path.join(configDict['rootDir'], configDict['sampleInfoFile']), index_col=0)

In [ ]:
sampleInfo

In [ ]:
colTemp = sampleInfo.columns
colTemp = [x.replace(' ','.') for x in colTemp]
colTemp = [x.replace('-','.') for x in colTemp]
sampleInfo.columns = colTemp


sampleInfo = sampleInfo.loc[:,sampleOrder]

In [ ]:
sampleInfo

In [ ]:
# Substrate, Diff, Size, Morphology, Day

In [ ]:
# factors = input('enter factors to be used for groups to check probe expression. separate multiple factors by a comma')

factors = 'Substrate, Diff, Size, Morphology, Day'

In [ ]:
factors = factors.split(',')
factors = [f.strip() for f in factors]

In [ ]:

# # factors = ['Sample_Type', 'Patient', 'Broad_classification']
# factors = ['Broad_classification']
# factors

In [ ]:
# generate a dictionary to hold factors for group analysis
factorDict = {}
factorDict2 = {}

for f in factors:
    entries = list(set(sampleInfo.loc[f].values))
    print(entries)
    factorDict[f] = entries
    factorDict2[f] = {}
    for e in entries:
        factorDict2[f][e] = sampleInfo.columns[sampleInfo.loc[f] == e]
    

In [ ]:
factorDict.keys()

In [ ]:
factorDict2

# Visualise Nanostring Norm results and choose samples to be kept for final normalisation

In [ ]:
configDict

In [ ]:
QCDataDF

In [ ]:
files = os.listdir(os.path.join(normDir, 'NSNorm'))
files = sorted(files)
# # print(len(files))
# # print(files)

# fileIdx = 0
# tempFile = os.path.join(normDir, 'NSNorm',files[fileIdx])

# print(tempFile)
# tempDF = pd.read_csv(os.path.join(normDir, 'NSNorm',files[fileIdx]), index_col=0)


In [ ]:
probeOrder

In [ ]:
width = 6
height = 14


fig, axs = plt.subplots(height, width, figsize=[40,25])
# fig.suptitle('Nanostring Normalisation heatmaps')

for y in range(height):
    for x in range(width):
        fileIdx = x + y*width
        tempDF = pd.read_csv(os.path.join(normDir, 'NSNorm',files[fileIdx]), index_col=0)
#         axs[y][x].matshow(np.log2(tempDF + 1), aspect = 'auto', cmap='coolwarm')
        axs[y][x].matshow(np.log2(tempDF.loc[probeOrder,sampleOrder] + 1), cmap='coolwarm')
        axs[y][x].set_xticks([])
        axs[y][x].set_yticks([])
        axs[y][x].set_title(files[fileIdx][:-4])
    
# plt.tight_layout()
# fig.show()
fig.savefig('NSNorm.png')


# Threshold data

In [ ]:
controlSet = set(['HYB-NEG', 'HYB-POS', 'Rb IgG', 'Ms IgG2a', 'Ms IgG1'])

### Threshold probes and drop low or null expressing probes

In [ ]:
# ToDo: Add in a variable called  thresholdData to hold data for comparison at thresholding stage. This will be read from one of the normalised files.

In [ ]:
thresholdData = pd.read_csv(os.path.join(normDir, 'NSNorm',files[51]), index_col=0)
thresholdData.columns = [x.replace(' ','.') for x in thresholdData.columns]
thresholdData.columns = [x.replace('-','.') for x in thresholdData.columns]



In [ ]:
threshold = 1.5
expPropCutOff = 0.5
SamplePropCutOff = 0.5

dropList = []

dropSetTemp = []

for f in factorDict2.keys():
    print(f)
    for g in factorDict2[f].keys():
        print(g)
        # print(factorDict2[f][g])
        groupLen = len(factorDict2[f][g])
        # print(groupLen)
        passThreshold = (thresholdData[factorDict2[f][g]]>threshold).sum(axis=1)
        # print(passThreshold)
        passThreshProp = (passThreshold/groupLen) < expPropCutOff
        # print(passThreshProp)
        failIdx = thresholdData.index[passThreshProp]
        print(failIdx)
        print(len(failIdx))

        if (len(failIdx) > 0):
            dropList.extend(list(failIdx))
            dropSetTemp.append(list(failIdx))
        # print(thresholdData.index[((thresholdData[factorDict2[f][g]]>2).sum(axis=1)/groupLen) < 0.5])

dropList = list(set(dropList) - controlSet)
print(dropList)
print(dropSetTemp)

# dropSet = set(tuple(x) for x in dropSetTemp)

In [ ]:
dropSet = set(dropSetTemp[0])

for x in range(1,len(dropSetTemp)):
    print(x)
    dropSet = dropSet & set(dropSetTemp[1])

dropSet = dropSet - controlSet
# dropSet
dropList = list(dropSet)
# dropList
QCDataDF = QCDataDF.drop(index=dropList)

In [ ]:
codeClass = codeClass[QCDataDF.index]

In [ ]:
codeClass

In [ ]:
print(len(dropList))
print(dropList)


### Threshold samples and drop low or null expressing samples

In [ ]:
dropSamples = list(QCDataDF[QCDataDF.columns[1:]].T[(QCDataDF[QCDataDF.columns[1:]]>threshold).sum(axis=0)/len(QCDataDF.index) < SamplePropCutOff].index)
print('dropSamples : ')
print(dropSamples)
QCDataDF = QCDataDF.drop(labels=dropSamples, axis=1)
# QCDataDF = QCDataDF.drop(index=dropSamples)
# QCDataDF

In [ ]:
QCDataDF

- export data after dropping probes and samples due to low expression
- re-run NSNorm
- Continue with viewing data and QC of normalised data



In [ ]:
# QCDataDF = pd.concat([codeClass,QCDataDF], axis=1)


In [ ]:
QCDataDF.index.name = 'Name'


In [ ]:
QCDataDF

In [ ]:
# ToDo: Ask for project name interactively
project = configDict['projectName']

qcDropCSV = 'QC_' + project + '_RUV_Dropped.csv'

writeOutput = True
if writeOutput:
    QCDataDF.to_csv(os.path.join(normDir, qcDropCSV))
writeOutput= False

In [ ]:
cmd = 'Rscript NSNorm.R -d ' + normDir + ' -s NSNormDropped -f ' + qcDropCSV
print(cmd)
os.system(cmd)

In [ ]:
files = os.listdir(os.path.join(normDir, 'NSNormDropped'))
files = sorted(files)


In [ ]:
width = 6
height = 14

fig, axs = plt.subplots(height, width, figsize=[40,25])
# fig.suptitle('Nanostring Normalisation heatmaps')
for y in range(height):
    for x in range(width):
        fileIdx = x + y*width
        tempDF = pd.read_csv(os.path.join(normDir, 'NSNormDropped',files[fileIdx]), index_col=0)
#         axs[y][x].matshow(np.log2(tempDF + 1), aspect = 'auto', cmap='coolwarm')
        # axs[y][x].matshow(np.log2(tempDF.loc[probeOrder,sampleOrder] + 1), cmap='coolwarm')
        axs[y][x].matshow(np.log2(tempDF + 1), cmap='coolwarm')
        axs[y][x].set_xticks([])
        axs[y][x].set_yticks([])
        axs[y][x].set_title(files[fileIdx][:-4])
    
# plt.tight_layout()
# fig.show()
fig.savefig('NSNormDropped.png')

# working

In [ ]:
# Drop samples from sample info file

In [ ]:
# generate groups for EdgeR analysis

In [ ]:
# temp = noneMeanHKDF > 0 

In [ ]:
# noneMeanHKDF = pd.read_csv(os.path.join(normDir, 'NSNorm',files[27]), index_col=0)
noneMeanHKDF = pd.read_csv(os.path.join(normDir, 'NSNormDropped',files[27]), index_col=0)


In [ ]:
# noneMeanHKDF

In [ ]:
# groupedExpressedIndex = noneMeanHKDF.loc[probeOrder].loc[((noneMeanHKDF > 0 ).sum(axis = 1) / len(noneMeanHKDF.columns) > 0.33333)].index


In [ ]:
# groupedExpressedIndex

In [ ]:
# len(groupedExpressedIndex)

In [ ]:
QCData = pd.read_csv(os.path.join(normDir, QCDataFile), index_col=0)

QCData

# Run EdgeR analysis

Write comparisons to a text file that will be parsed by the r script

In form of factor.variable, factor.variable2, comparisonName
1 comparison per line

  make_option(c("-d", "--datadir"), type="character", default=NULL, 
              help="dataset file name", metavar="character"),
  make_option(c("-f", "--file"), type="character", default=NULL, 
              help="dataset file name", metavar="character"),
  make_option(c("-e", "--exportdir"), type="character", default="NSNorm", 
              help="dataset file name", metavar="character"),
  make_option(c("-i", "--sampleinfo"), type="character", default=NULL, 
              help="dataset file name", metavar="character")

### extract potential factors for analysis from info file

In [ ]:
# groups = ['Broad_classification']

groups = [
    'Substrate',
    'Substrate.Day.Diff',
    'Substrate.Day.Diff.Layer',
    'Substrate.Day.Diff.Morphology.Layer',
    'Substrate.Day.Diff.Size.Layer',
    'Substrate.Day.Diff.Size.Morphology.Layer'
]

In [ ]:
# ['Sample_Type', 'Patient', 'Broad_classification']

### extract potential groups based on factors

### Show group numbers for each of the comparisons

In [ ]:
# comps = [
#     [
#         'Hydrogel - Spheroid'
#     ],
#     [
#         'Hydrogel.BDNF - Hydrogel.SD',
#         'Hydrogel.Dream - Hydrogel.SD',
#         'Hydrogel.PDGF - Hydrogel.SD',
#         'Spheroid.Hep - Spheroid.SD'    
#     ],
#     [
#         'Hydrogel.SD.Outer - Hydrogel.SD.Inner',
#         'Hydrogel.BDNF.Outer - Hydrogel.BDNF.Inner',
#         'Hydrogel.Dream.Outer - Hydrogel.Dream.Inner',
#         'Hydrogel.PDGF.Outer - Hydrogel.PDGF.Inner',
#         'Spheroid.SD.Outer - Spheroid.SD.Inner',
#         'Spheroid.Hep.Outer - Spheroid.Hep.Inner'
#     ],
# ]

# compNames = []
# for g in comps:
#     ctemp = []
#     for c in g:
#         c = c.replace('.','_')
#         c = c.replace(' - ','_vs_')
#         ctemp.append(c)
#     compNames.append(ctemp)
    

# compNames

In [ ]:
# comps = [
#     [
#         'Secondary - Primary'
#     ]
# ]

# compNames = []
# for g in comps:
#     ctemp = []
#     for c in g:
#         c = c.replace('.','_')
#         c = c.replace(' - ','_vs_')
#         ctemp.append(c)
#     compNames.append(ctemp)
    

# compNames

In [ ]:
comps = [
    [
        # 'Substrate',
        'Hydrogel - Spheroid' 
    ],
    [
        # 'Substrate.Day.Diff',        
        'Spheroid.D7.Hep - Spheroid.D7.SD', #1
        'Hydrogel.D14.SD - Hydrogel.D7.SD', #2

        'Hydrogel.D14.Dream - Hydrogel.D14.SD', #3
        'Hydrogel.D14.PDGF - Hydrogel.D14.SD', #4
        'Hydrogel.D14.BDNF - Hydrogel.D14.SD', #5
        
        'Hydrogel.D14.PDGF - Hydrogel.D14.Dream', #6
        'Hydrogel.D14.BDNF - Hydrogel.D14.Dream', #7
        
        'Hydrogel.D14.BDNF - Hydrogel.D14.PDGF' #8
    ],
    [
        # 'Substrate.Day.Diff.Layer',
        # 'Spheroid.D7.SD.Outer - Spheroid.D7.SD.Inner',
        'Spheroid.D7.SD.Inner - Spheroid.D7.SD.Outer', #1
        # 'Spheroid.D7.Hep.Outer - Spheroid.D7.Hep.Inner',
        'Spheroid.D7.Hep.Inner - Spheroid.D7.Hep.Outer', #2
        
        'Spheroid.D7.Hep.Inner - Spheroid.D7.SD.Inner', #3
        'Spheroid.D7.Hep.Outer - Spheroid.D7.SD.Outer', #4

        'Hydrogel.D7.SD.Inner - Hydrogel.D7.SD.Outer', #5
        # 'Hydrogel.D14.SD.Outer - Hydrogel.D14.SD.Inner',
        'Hydrogel.D14.SD.Inner - Hydrogel.D14.SD.Outer', #6
        # 'Hydrogel.D14.BDNF.Outer - Hydrogel.D14.BDNF.Inner',
        # 'Hydrogel.D14.Dream.Outer - Hydrogel.D14.Dream.Inner',
        # 'Hydrogel.D14.PDGF.Outer - Hydrogel.D14.PDGF.Inner',
        'Hydrogel.D14.Dream.Inner - Hydrogel.D14.Dream.Outer', #7
        'Hydrogel.D14.PDGF.Inner - Hydrogel.D14.PDGF.Outer', #8
        'Hydrogel.D14.BDNF.Inner - Hydrogel.D14.BDNF.Outer', #9

        
        'Hydrogel.D14.Dream.Inner - Hydrogel.D14.SD.Inner', #10
        'Hydrogel.D14.PDGF.Inner - Hydrogel.D14.SD.Inner', #11
        'Hydrogel.D14.BDNF.Inner - Hydrogel.D14.SD.Inner', #12

        'Hydrogel.D14.Dream.Outer - Hydrogel.D14.SD.Outer', #13
        'Hydrogel.D14.PDGF.Outer - Hydrogel.D14.SD.Outer', #14
        'Hydrogel.D14.BDNF.Outer - Hydrogel.D14.SD.Outer', #15

        
        'Hydrogel.D14.PDGF.Inner - Hydrogel.D14.Dream.Inner', #16
        'Hydrogel.D14.BDNF.Inner - Hydrogel.D14.Dream.Inner', #17

        'Hydrogel.D14.PDGF.Outer - Hydrogel.D14.Dream.Outer', #18
        'Hydrogel.D14.BDNF.Outer - Hydrogel.D14.Dream.Outer', #19

        
        'Hydrogel.D14.BDNF.Inner - Hydrogel.D14.PDGF.Inner', #20

        'Hydrogel.D14.BDNF.Outer - Hydrogel.D14.PDGF.Outer' #21
    ],
    [
        # 'Substrate.Day.Diff.Morph.Layer',
        'Hydrogel.D7.SD.Round.Inner - Hydrogel.D7.SD.Round.Outer', #1
        'Hydrogel.D7.SD.Diff.Inner - Hydrogel.D7.SD.Diff.Outer', #2
        'Hydrogel.D7.SD.Diff.Inner - Hydrogel.D7.SD.Round.Inner', #3
        'Hydrogel.D7.SD.Diff.Outer - Hydrogel.D7.SD.Round.Outer' #4
    ],
    [
        # 'Substrate.Day.Diff.Size.Layer'
        ## D7 SD
        ### In - out
        'Hydrogel.D7.SD.Small.Inner - Hydrogel.D7.SD.Small.Outer', #1
        'Hydrogel.D7.SD.Large.Inner - Hydrogel.D7.SD.Large.Outer', #2
        ### sml - lge
        'Hydrogel.D7.SD.Small.Inner - Hydrogel.D7.SD.Large.Inner', #3
        'Hydrogel.D7.SD.Small.Outer - Hydrogel.D7.SD.Large.Outer', #4
        
        ## D14 SD
        ### In - out
        'Hydrogel.D14.SD.Small.Inner - Hydrogel.D14.SD.Small.Outer', #1
        'Hydrogel.D14.SD.Large.Inner - Hydrogel.D14.SD.Large.Outer', #2
        ### sml - lge
        'Hydrogel.D14.SD.Small.Inner - Hydrogel.D14.SD.Large.Inner', #3
        'Hydrogel.D14.SD.Small.Outer - Hydrogel.D14.SD.Large.Outer', #4

        ## Dream
        ### In - out
        'Hydrogel.D14.Dream.Small.Inner - Hydrogel.D14.Dream.Small.Outer', #1
        # 'Hydrogel.D14.Dream.Large.Inner - Hydrogel.D14.Dream.Large.Outer',
        ### sml - lge
        # 'Hydrogel.D14.Dream.Small.Inner - Hydrogel.D14.Dream.Large.Inner',
        'Hydrogel.D14.Dream.Small.Outer - Hydrogel.D14.Dream.Large.Outer', #2
        
        ## PDGF
        ### In - out
        'Hydrogel.D14.PDGF.Small.Inner - Hydrogel.D14.PDGF.Small.Outer', #1
        # 'Hydrogel.D14.PDGF.Large.Inner - Hydrogel.D14.PDGF.Large.Outer',
        ### sml - lge
        # 'Hydrogel.D14.PDGF.Small.Inner - Hydrogel.D14.PDGF.Large.Inner',
        'Hydrogel.D14.PDGF.Small.Outer - Hydrogel.D14.PDGF.Large.Outer', #2
        
        ## BDNF
        ### In - out
        'Hydrogel.D14.BDNF.Small.Inner - Hydrogel.D14.BDNF.Small.Outer', #1
        # 'Hydrogel.D14.BDNF.Large.Inner - Hydrogel.D14.BDNF.Large.Outer',
        ### sml - lge
        # 'Hydrogel.D14.BDNF.Small.Inner - Hydrogel.D14.BDNF.Large.Inner',
        'Hydrogel.D14.BDNF.Small.Outer - Hydrogel.D14.BDNF.Large.Outer' #2
        
    ],
    [
        # 'Substrate.Day.Diff.Size.Morph.Layer'
        ## D7 SD
        ### In - out
        'Hydrogel.D7.SD.Small.Round.Inner - Hydrogel.D7.SD.Small.Round.Outer', #1
        'Hydrogel.D7.SD.Large.Round.Inner - Hydrogel.D7.SD.Large.Round.Outer', #2
        'Hydrogel.D7.SD.Small.Diff.Inner - Hydrogel.D7.SD.Small.Diff.Outer', #3
        # 'Hydrogel.D7.SD.Large.Diff.Inner - Hydrogel.D7.SD.Large.Diff.Outer',

    
        ### round - diff
        'Hydrogel.D7.SD.Small.Round.Inner - Hydrogel.D7.SD.Small.Diff.Inner', #4
        'Hydrogel.D7.SD.Small.Round.Outer - Hydrogel.D7.SD.Small.Diff.Outer', #5
        # 'Hydrogel.D7.SD.Large.Round.Inner - Hydrogel.D7.SD.Large.Diff.Inner',
        'Hydrogel.D7.SD.Large.Round.Outer - Hydrogel.D7.SD.Large.Diff.Outer' #6

    ]
]

compNames = []
for g in comps:
    ctemp = []
    for c in g:
        c = c.replace('.','_')
        c = c.replace(' - ','_vs_')
        ctemp.append(c)
    compNames.append(ctemp)
    

compNames

In [ ]:
# configDict['rootDir']

In [ ]:
normPath = os.path.join('Normalisation','NSNormDropped')
print(normPath)

normFile = 'NanoStringNorm_49_none_none_low.cv.geo.mean.csv'
print(normFile)

exportdir = 'EdgeR'
# runname = subfolder
sampleinfoFile = 'sampleInfo_with_Wells.csv'

### Set up config file for EdgeR

In [ ]:
os.getcwd()

In [ ]:
outFile = 'EdgeR_Config.txt'

outLines = []
for g in range(len(groups)):
    groupLine = 'GROUP:' + groups[g]
    outLines.append(groupLine)
    
    compLine = 'COMPARISON:'
    compNameLine = 'COMP_NAME:'
    for c in range(len(comps[g])):
        compLine += comps[g][c]
        compLine += ','
        compNameLine += compNames[g][c]
        compNameLine += ','
    
    outLines.append(compLine)
    outLines.append(compNameLine)

with open(outFile, 'w') as o:
    o.write('\n'.join(outLines))
    o.write('\n')

In [ ]:
## Need to handle dropped samples that may still be present in sampleinfoFile. Could be handled with another file, or by removing samples within R.


In [ ]:

# cmd = 'Rscript EdgeR.R -c ' + os.getcwd() + ' -d ' + configDict['rootDir'] + ' -n ' + normPath + ' -f ' + normFile + ' -e ' + exportdir + ' -r ' + runname  + ' -i ' + sampleinfoFile
cmd = 'Rscript ../DSP_EDA_Protein/EdgeR.R -c ' + os.getcwd() + ' -d ' + configDict['rootDir'] + ' -n ' + normPath + ' -f ' + normFile + ' -e ' + exportdir + ' -i ' + sampleinfoFile
print(cmd)
os.system(cmd)

# Convert MD Plots to Volcano Plots

In [ ]:
# Currently done in a separate notebook

In [ ]:
os.path.join(configDict['rootDir'],exportdir)

In [ ]:
dataPath = os.path.join(configDict['rootDir'],exportdir,)

filesMaster = []

for root, folder, files in os.walk(dataPath):
    print(root)
    print(folder)
    print(files)

    files = [os.path.join(root, f) for f in files if (f.endswith('.csv') and f.startswith('MD_plot'))]
    filesMaster.extend(files)
     

In [ ]:

sigGenes = []

TMESigGenes = []
TumourSigGenes = []

for file in filesMaster:
    subGenes = volcanoPlotpVal(dataPath, file)
    subGenes = volcanoPlot(dataPath, file)
    sigGenes.extend(subGenes)
    
    
    # if 'Tumour' in file:
    #     TumourSigGenes.extend(subGenes)
    # elif 'TME' in file:
    #     TMESigGenes.extend(subGenes)
    
# print(sigGenes)
# print(set(sigGenes))

    
sigGenes = list(set(sigGenes))
    
    

In [ ]:
break

In [ ]:
# dataPath = '../../../Nanostring/projects/.../DSP_Protein_Data/'
dataPath = configDict['rootDir']
data = pd.read_csv(os.path.join(dataPath,'HK_Geo_Mean_Normalised.csv'), index_col = 0)
probeFilter = pd.read_csv(os.path.join(dataPath,'Probe_Filter.csv'), index_col = 0)
sampleInfo = pd.read_csv(os.path.join(dataPath,'Sample_Info.csv'), index_col = 0)

# dataPath = '../../../Nanostring/projects/.../EdgeR/EdgeR_normData.tsv'
dataPath = os.path.join(configDict['rootDir'], normPath, normFile)

data = pd.read_csv(dataPath, index_col = 0)

# dataPath = '../../../Nanostring/projects/.../DSP_Protein_Data/'
dataPath = configDict['rootDir']


In [ ]:
data

In [ ]:
sampleInfo.columns

In [ ]:
wb = load_workbook(os.path.join(dataPath,'Annotation template file-1a_wells_02.xlsx'))

print(wb.sheetnames)



In [ ]:
ws = wb['Annotation template']

segments = [[y.value for y in x] for x in ws[ws.calculate_dimension()]]
df = pd.DataFrame(segments)


rowLabels = df.iloc[1:,0]
colLabels = df.iloc[0,1:]
annotations = df.iloc[1:,1:]
rowLabels += '_'
rowLabels += df.iloc[1:,1]
rowLabels += '_Full ROI'



In [ ]:
rowLabels

In [ ]:
colLabels

In [ ]:
annotations.values

In [ ]:
sampleAnnotations = pd.DataFrame(annotations.values, index=rowLabels, columns=colLabels)

# sampleAnnotations = sampleAnnotations.T
# sampleAnnotations.set_index(0, drop=True, inplace=True)
# sampleAnnotations = sampleAnnotations.T
# sampleAnnotations.set_index('Scan name', drop=True, inplace=True)
# sampleAnnotations = sampleAnnotations.T



In [ ]:
sampleAnnotations

In [ ]:
sampleAnnotations = sampleAnnotations.join(sampleInfo.T,lsuffix='Drop').T

In [ ]:
sampleAnnotations.drop(labels=[x for x in sampleAnnotations.index if x.endswith('Drop')], inplace=True)

In [ ]:
sampleInfo = sampleAnnotations

In [ ]:
sampleInfo

In [ ]:
def standardize_data(arr):
         
    '''
    This function standardize an array, its substracts mean value, 
    and then divide the standard deviation.
    
    param 1: array 
    return: standardized array
    '''    
    rows, columns = arr.shape
    
    standardizedArray = np.zeros(shape=(rows, columns))
    tempArray = np.zeros(rows)
    
    for column in range(columns):
        
        mean = np.mean(X[:,column])
        std = np.std(X[:,column])
        tempArray = np.empty(0)
        
        for element in X[:,column]:
            
            tempArray = np.append(tempArray, ((element - mean) / std))
 
        standardizedArray[:,column] = tempArray
    
    return standardizedArray

In [ ]:
# Standardizing data

### I'm not sure that the transpose is what i want here. The data is the wrong shape and pc's seems to be being calculated for proteins instead of samples
X = endogNorm.transpose().values
# X = endogNorm.values
### Try to get pc's for samples
# X = endogNorm.values

## ??? With transpose makes PCA for effect of variables on smaples, without makes PCA for effects of variables on protein expression levels
###^^^ Maybe the other way round?



X_cols = endogNorm.columns
print(X_cols.shape)
y = endogNorm.index
print(y.shape)
X = standardize_data(X)
print(X.shape)


In [ ]:
# Calculating the covariance matrix

covariance_matrix = np.cov(X.T)
# covariance_matrix = np.cov(X)



print(covariance_matrix.shape)


In [ ]:
# Using np.linalg.eig function

eigen_values, eigen_vectors = np.linalg.eig(covariance_matrix)
print("Eigenvector: \n",eigen_vectors,"\n")
print("Eigenvalues: \n", eigen_values, "\n")
print(eigen_vectors.shape)

In [ ]:
eigenDF = pd.DataFrame(eigen_vectors, index=[endogNorm.index], columns=[endogNorm.index])
# eigenDF = pd.DataFrame(eigen_vectors, index=[endogNorm.columns], columns=[endogNorm.columns])

In [ ]:
# Calculating the explained variance on each of components


variance_explained = []
for i in eigen_values:
     variance_explained.append((i/sum(eigen_values))*100)
        
print(variance_explained)

In [ ]:
# Identifying components that explain at least 95%

cumulative_variance_explained = np.cumsum(variance_explained)
print(cumulative_variance_explained)


In [ ]:
cumulative_variance_explained = [np.float64(x) for x in cumulative_variance_explained]

In [ ]:
cumulative_variance_explained[0].dtype

In [ ]:
# Visualizing the eigenvalues and finding the "elbow" in the graphic


sns.lineplot(x = [i for i in range(len(cumulative_variance_explained))], y=cumulative_variance_explained)
# plt.xlabel("Number of components")
# plt.ylabel("Cumulative explained variance")
# plt.title("Explained variance vs Number of components")



# ToDo:
# Add lines for 95% variance, and number of components describing at least 95% of variance



In [ ]:
# Using two first components (because those explain more than 95%)

projection_matrix = (eigen_vectors.T[:][:50]).T
print(projection_matrix)

In [ ]:
colours = ['g' if x.split('_')[-1] == 'Tumour' else 'r' for x in X_cols]
colours = ['g' if x.split('_')[-1] == 'Tumour' else 'r' if x.split('_')[-1] == 'Immune' else 'purple' for x in X_cols]


In [ ]:
# I dont think this gives relevant info. projection matrix needs to be combined with original data to see effects of components on patients

plt.scatter([x[0] for x in projection_matrix], [x[1] for x in projection_matrix])#, c=colours)

projection_matrix.shape

In [ ]:
plt.scatter([x[2] for x in projection_matrix], [x[1] for x in projection_matrix])#, c=colours)


In [ ]:
plt.scatter([x[2] for x in projection_matrix], [x[3] for x in projection_matrix])#, c=colours)


In [ ]:
# Getting the product of original standardized X and the eigenvectors 


X_pca = X.dot(projection_matrix)
print(X_pca)

In [ ]:
X_pca.shape

In [ ]:
sampleInfo

In [ ]:
# tagDF = pd.DataFrame( data=[x.split(',') for x in sampleInfo.loc['Segment tags']], index=sampleInfo.columns, columns=['Obese','Arth','Patellar'])

In [ ]:
# tagDF

In [ ]:
# sampleInfo = pd.concat([sampleInfo,tagDF.T])

In [ ]:
sampleInfo

In [ ]:
inforSortedIndex = sampleInfo.sort_values(by=['Obese','arthritis','TMA_Core'], axis=1).columns

In [ ]:
### Add umap / tSNE analysis here